In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
import os
import cv2
import glob

Using TensorFlow backend.


In [24]:
train_dir = 'cnn_data_train'
train_path = os.path.join(train_dir, '*g')

train_imgs = glob.glob(train_path)
train_labels = [int(name[-5]) for name in train_imgs]
train_imgs = np.array([np.array(cv2.imread(img)) for img in train_imgs])

In [25]:
val_dir = 'cnn_data_val'
val_path = os.path.join(val_dir, '*g')

val_imgs = glob.glob(val_path)
val_labels = [int(name[-5]) for name in val_imgs]
val_imgs = np.array([np.array(cv2.imread(img)) for img in val_imgs])

In [26]:
test_dir = 'cnn_data_test'
test_path = os.path.join(val_dir, '*g')

test_imgs = glob.glob(test_path)
test_labels = np.array([int(name[-5]) for name in test_imgs])
test_imgs = np.array([np.array(cv2.imread(img)) for img in test_imgs])

For image loading and preprocessing see:
https://keras.io/preprocessing/image/

In [27]:
batch_size = 128
num_classes = 2
epochs = 10

# input image dimensions
img_rows, img_cols = 60, 60

print('train_imgs shape:', train_imgs.shape)
print(train_imgs.shape[0], 'train samples')
print(test_imgs.shape[0], 'test samples')

# convert class vectors to binary class matrices
train_label = keras.utils.to_categorical(train_labels, num_classes)
val_label = keras.utils.to_categorical(val_labels, num_classes)
test_label = keras.utils.to_categorical(test_labels, num_classes)

train_imgs shape: (1011, 60, 60, 3)
1011 train samples
217 test samples


The model used by us:

In [28]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(img_rows,img_cols,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [30]:
model2.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model2.fit(train_imgs, train_label,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split=.2)
score = model2.evaluate(test_imgs, test_label, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 808 samples, validate on 203 samples
Epoch 1/10
808/808 [==============================] - 4s 5ms/step - loss: 0.1000 - acc: 0.9777 - val_loss: 1.1399 - val_acc: 0.7488
Epoch 2/10
808/808 [==============================] - 3s 4ms/step - loss: 0.0276 - acc: 0.9988 - val_loss: 1.2826 - val_acc: 0.7586
Epoch 3/10
808/808 [==============================] - 3s 4ms/step - loss: 0.0220 - acc: 1.0000 - val_loss: 1.4570 - val_acc: 0.7537
Epoch 4/10
808/808 [==============================] - 3s 4ms/step - loss: 0.0213 - acc: 0.9988 - val_loss: 1.4172 - val_acc: 0.7586
Epoch 5/10
808/808 [==============================] - 3s 4ms/step - loss: 0.0189 - acc: 0.9988 - val_loss: 1.5409 - val_acc: 0.7586
Epoch 6/10
808/808 [==============================] - 3s 4ms/step - loss: 0.0161 - acc: 0.9988 - val_loss: 1.5309 - val_acc: 0.7685
Epoch 7/10
808/808 [==============================] - 3s 4ms/step - loss: 0.0301 - acc: 0.9913 - val_loss: 1.6427 - val_acc: 0.7488
Epoch 8/10
808/808 [==========

The model used by DNT:

In [21]:
model2 = Sequential()
model2.add(Conv2D(4, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=(img_rows,img_cols,3)))
model2.add(Conv2D(5, (3, 3), activation='relu'))
model2.add(Dropout(0.3))
model2.add(Flatten())
model2.add(Dense(16, activation='relu', input_shape=(24*24*5,)))
model2.add(Dense(num_classes, activation='softmax'))

The model used by B-Human

In [ ]:
# B-Human CNN komt hier